In [1]:
#learning rate decay : 초기에는 빠르게 나중에는 천천히

import tensorflow as tf
import math
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
tf.set_random_seed(0)

mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

X = tf.placeholder(tf.float32, [None, 28,28,1])
Y_ = tf.placeholder(tf.float32, [None, 10])
lr = tf.placeholder(tf.float32)

XX = tf.reshape(X, [-1, 784])

W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
b1 = tf.Variable(tf.ones([200])/10)
Y1 = tf.nn.relu(tf.matmul(XX, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
b2 = tf.Variable(tf.ones([100])/10)
Y2 = tf.nn.relu(tf.matmul(Y1, W2) + b2)

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
b3 = tf.Variable(tf.ones([60])/10)
Y3 = tf.nn.relu(tf.matmul(Y2, W3) + b3)

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
b4 = tf.Variable(tf.ones([30])/10)
Y4 = tf.nn.relu(tf.matmul(Y3, W4) + b4)

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
b5 = tf.Variable(tf.zeros([10]))
Y5 = tf.matmul(Y4, W5) + b5

Y = tf.nn.softmax(Y5)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y5, labels=Y_)) * 100.0

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(0.0005).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [3]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
max_accuracy = 0.0

def training_step(i, update_test_data, update_train_data):
    
    global max_accuracy
    batch_X, batch_Y = mnist.train.next_batch(100)
    
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) *math.exp(-i/decay_speed)
    
    if(update_train_data):
        a, c = sess.run([accuracy, cross_entropy], feed_dict = {X:batch_X, Y_:batch_Y})
        print(str(i) + ": acrcuracy:" + str(a) + " loss: " + str(c) + " (lr:" + str(learning_rate) + ")")
    
    if(update_test_data):
        a, c = sess.run([accuracy, cross_entropy], feed_dict = {X:mnist.test.images, Y_:mnist.test.labels})
        if(a>max_accuracy):
            max_accuracy = a
        print(str(i) + ": ******* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ******* test accuracy:" + str(a) + " test loss: " + str(c))
    sess.run (train_step, feed_dict={X: batch_X, Y_: batch_Y, lr: learning_rate})

for i in range(2000+1):
    training_step(i, i%50==0, i%10 == 0)
print("max test accuracy: " + str(max_accuracy))

0: acrcuracy:0.08 loss: 232.96516 (lr:0.003)
0: ******* epoch 1 ******* test accuracy:0.0772 test loss: 232.38445
10: acrcuracy:0.35 loss: 221.72885 (lr:0.0029855361896587787)
20: acrcuracy:0.49 loss: 208.88753 (lr:0.0029711445178725875)
30: acrcuracy:0.52 loss: 185.31119 (lr:0.0029568246248488817)
40: acrcuracy:0.53 loss: 151.2927 (lr:0.0029425761525895904)
50: acrcuracy:0.65 loss: 113.71137 (lr:0.0029283987448821647)
50: ******* epoch 1 ******* test accuracy:0.6377 test loss: 115.7415
60: acrcuracy:0.75 loss: 97.45021 (lr:0.0029142920472906737)
70: acrcuracy:0.83 loss: 83.583084 (lr:0.0029002557071469426)
80: acrcuracy:0.8 loss: 65.41127 (lr:0.0028862893735417373)
90: acrcuracy:0.8 loss: 61.799908 (lr:0.0028723926973159898)
100: acrcuracy:0.92 loss: 40.146263 (lr:0.0028585653310520707)
100: ******* epoch 1 ******* test accuracy:0.8391 test loss: 54.100536
110: acrcuracy:0.83 loss: 55.676525 (lr:0.002844806929065103)
120: acrcuracy:0.83 loss: 61.76005 (lr:0.0028311171473943213)
130: a

1080: acrcuracy:0.9 loss: 28.150637 (lr:0.0017899699318845701)
1090: acrcuracy:0.97 loss: 9.911093 (lr:0.0017815411716855546)
1100: acrcuracy:0.98 loss: 11.542522 (lr:0.0017731544501034114)
1100: ******* epoch 2 ******* test accuracy:0.9586 test loss: 13.762108
1110: acrcuracy:0.91 loss: 27.685589 (lr:0.0017648095574696644)
1120: acrcuracy:0.94 loss: 15.447328 (lr:0.0017565062851615633)
1130: acrcuracy:0.99 loss: 5.246856 (lr:0.0017482444255968678)
1140: acrcuracy:0.91 loss: 22.327898 (lr:0.0017400237722286578)
1150: acrcuracy:0.97 loss: 11.555874 (lr:0.0017318441195401718)
1150: ******* epoch 2 ******* test accuracy:0.9605 test loss: 13.209354
1160: acrcuracy:0.95 loss: 9.442512 (lr:0.001723705263039666)
1170: acrcuracy:0.96 loss: 10.73392 (lr:0.0017156069992553045)
1180: acrcuracy:0.97 loss: 7.7784166 (lr:0.0017075491257300707)
1190: acrcuracy:0.96 loss: 10.338802 (lr:0.0016995314410167067)
1200: acrcuracy:0.97 loss: 8.1813135 (lr:0.001691553744672677)
1200: ******* epoch 3 ******* t